<a href="https://colab.research.google.com/github/oli2tup/CE301/blob/main/Malaria_classification_using_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, MaxPool2D

from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator


print(tf.__version__)

import numpy as np
import matplotlib.pyplot as plt

2.6.0


In [17]:
!git clone https://github.com/oli2tup/CE301.git


fatal: destination path 'CE301' already exists and is not an empty directory.


In [18]:
img_width = 100
img_height = 100

In [19]:
datagen = ImageDataGenerator(rescale=1/255.0, validation_split=0.2) # using 20% of data as validation

In [20]:
train_data_generator = datagen.flow_from_directory(
    directory ='/content/CE301/cell_images', target_size =(img_width, img_height),
    class_mode = 'binary',
    batch_size = 16,
    subset = 'training'
) # Training subset

Found 22048 images belonging to 2 classes.


In [21]:
validation_data_generator = datagen.flow_from_directory(
    directory ='/content/CE301/cell_images', target_size =(img_width, img_height),
    class_mode = 'binary',
    batch_size = 16,
    subset = 'validation'
) # Validation subset

Found 5510 images belonging to 2 classes.


In [15]:
train_data_generator.labels

array([0, 0, 0, ..., 1, 1, 1], dtype=int32)

In [16]:
validation_data_generator.labels

array([0, 0, 0, ..., 1, 1, 1], dtype=int32)

In [ ]:
class ResidualUnit(keras.layers.Layer):
  def __init__(self, filters, strides=1, activation="relu", **kwargs):
    super().__init__(**kwargs)
    self.activation = keras.activations.get(activation)
    self.main_layers = [
      keras.layers.Conv2D(filters, 3, strides=strides,
        padding="same", use_bias=False),
      keras.layers.BatchNormalization(),
      self.activation,
      keras.layers.Conv2D(filters, 3, strides=1,
        padding="same", use_bias=False),
      keras.layers.BatchNormalization()]
    self.skip_layers = []
    if strides > 1:
      self.skip_layers = [
        keras.layers.Conv2D(filters, 1, strides=strides,
          padding="same", use_bias=False),
        keras.layers.BatchNormalization()]
  def call(self, inputs):
    Z  = inputs
    for layer in self.main_layers:
      Z = layer(Z)
    skip_Z = inputs
    for layer in self.skip_layers:
      skip_Z = layer(skip_Z)
      return self.activation(Z + skip_Z)